In [1]:
import numpy as np
import itertools

In [21]:
# Function which checks whether the input state is a winning state.
# playerIdx = 1 means the player is 'X', playerIdx = 2 means the player is 'O'
def isWinningState(inState, playerIdx=1):
    
    # Check for horizontal win states
    idx1 = 0
    idx2 = 1
    idx3 = 2
    for i in xrange(3):
        if (inState[idx1] == inState[idx2] == inState[idx3]) and inState[idx1] == playerIdx:
            return True
        
        idx1 = idx1 + 3
        idx2 = idx2 + 3
        idx3 = idx3 + 3
    
    # Check for vertical win states
    idx1 = 0
    idx2 = 3
    idx3 = 6
    
    for i in xrange(3):
        if (inState[idx1] == inState[idx2] == inState[idx3]) and inState[idx1] == playerIdx:
            return True
        
        idx1 = idx1 + 1
        idx2 = idx2 + 1
        idx3 = idx3 + 1
    
    # Check for diagonal win state 1
    if (inState[0] == inState[4] == inState[8]) and inState[0] == playerIdx:
        return True
    
    # Check for diagonal win state 2
    if (inState[2] == inState[4] == inState[6]) and inState[2] == playerIdx:
        return True
    
    # If we got to here, no win states were found
    return False
            

In [42]:
# Checks if this is a drawn state.
# Note that this should be called AFTER checking if 
# this state is a winning state because this does no
# explicit win check
def isDrawn(state):
    for i in xrange(9):
        if state[i] == 0:
            return False
        
    # If we get to here then it is a drawn game
    return True

In [54]:
# Determines possible next states for the computer
def nextStates(state, playerIdx=1):
    nextStates = []
    stateList = list(state)
    for i in xrange(9):
        if stateList[i] == 0:
            newState = list(stateList)
            newState[i] = playerIdx
            nextStates.append(tuple(newState))
            
    return nextStates

In [40]:
# Takes a state as a tuple, e.g. (1, 0, 0, 1, 1, 0, 0, 0, 0)
# and prints out the appearance of the board, e.g:
#
#           X  |      |    
#           X  |   X  |    
#              |      |
def printBoardState(state):
    idx = 0
    lookup = {0: ' ', 1: 'X', 2: 'O'}
    for i in xrange(3):
        c0 = lookup[state[idx]]
        c1 = lookup[state[idx+1]]
        c2 = lookup[state[idx+2]]
        print c0 + '\t|\t' + c1 + '\t|\t' + c2
        idx = idx + 3

In [33]:
# Our computer will play player 1, i.e. it will play as 'X'
playerIdx = 1
# Create lookup table for all possible board states and set initial values to zero
V = {}
for combination in itertools.product(xrange(3), repeat=9):
    if (isWinningState(combination, playerIdx)):
        V[combination] = 1.0
    else:
        V[combination] = 0.0

In [43]:
nTrainingEpisodes = 1
stepSize = 0.1
exploreProbability = 0.1

for i in xrange(nTrainingEpisodes):
    # Initialise the game
    curState = (0,0,0,0,0,0,0,0,0)
    while True:
        futureStates = nextStates(curState)
        nFutureStates = len(futureStates)
        # Explore suboptimal states with some probability
        if np.random.uniform() < exploreProbability:
            # Select a random state to explore
            randomStateIdx = int(nFutureStates * np.random.uniform())
            curState = futureStates[randomStateIdx]
        else:
            # Iterate through future states and find best one
            # in terms of value function, V
            maxValue = V[futureStates[0]]
            maxIdx = 0
            for j in xrange(nFutureStates):
                if V[futureStates[j]] > maxValue:
                    maxValue = V[futureStates[j]]
                    maxIdx = j
            
            
        
        

IndentationError: expected an indented block (<ipython-input-43-0b01ee144634>, line 9)

In [47]:
x = (1,1,1)
y = list(x)
y[1] = 55
x = tuple(y)
x

(1, 55, 1)

In [55]:
possNextStates = nextStates((0,0,0,0,0,0,0,0,0))

In [56]:
len(possNextStates)
print possNextStates

[(1, 0, 0, 0, 0, 0, 0, 0, 0), (0, 1, 0, 0, 0, 0, 0, 0, 0), (0, 0, 1, 0, 0, 0, 0, 0, 0), (0, 0, 0, 1, 0, 0, 0, 0, 0), (0, 0, 0, 0, 1, 0, 0, 0, 0), (0, 0, 0, 0, 0, 1, 0, 0, 0), (0, 0, 0, 0, 0, 0, 1, 0, 0), (0, 0, 0, 0, 0, 0, 0, 1, 0), (0, 0, 0, 0, 0, 0, 0, 0, 1)]


In [61]:
uniformDist = np.random.uniform()

In [62]:
uniformDist

0.047265481263974674

In [180]:
int(9 * np.random.uniform())

8